# Dynamic Routing between Capsules for the MNIST Data-Set

## Introduction

We are basing our code on the model proposed in the paper [Dynamic Routing between capsules](https://arxiv.org/abs/1710.09829).

## Imports

In [9]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.examples.tutorials.mnist import input_data
np.random.seed(42)
tf.set_random_seed(42)

We used the tensorflow version 1.7.0

A lot of this code was inspired by the amazing Github of Aurélien Geron :
https://github.com/ageron/handson-ml/blob/master/extra_capsnets.ipynb

In [10]:
print(tf.__version__)

1.7.0


https://github.com/exelban/tensorflow-cifar-10/blob/master/include/model.py

## Data

We use the inbuilt MNIST data-set from Tensorflow and download it into the data_out folder. We do not train our model on other data than the normal MNIST train set.

In [11]:
restore_checkpoint = True
path = "./data_out/"

mnist = input_data.read_data_sets("/tmp/data/")

# Train Set
x_train = mnist.train.images
y_train = mnist.train.labels

print(x_train.shape)
print(y_train.shape)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000,)


### Rotated MNIST Data-Set

We used the following data sets from the LISA MNIST webpage: http://www.iro.umontreal.ca/~lisa/twiki/bin/view.cgi/Public/MnistVariations

- MNIST basic
- MNIST + background images
- MNIST + random background
- Rotated MNIST digits
- Rotated MNIST digits + background images

Put the downloaded data into a folder called data_transfo.

In [12]:
path_test = './data_transfo/'

Load the data into variables with numpy.loadtext .

In [13]:
test_normal = np.loadtxt(path_test + 'mnist_test.amat')
test_rotated = np.loadtxt(path_test + 'mnist_all_rotation_normalized_float_test.amat')
test_back_rand = np.loadtxt(path_test + 'mnist_background_random_test.amat')
test_back_img = np.loadtxt(path_test + 'mnist_background_images_test.amat')
test_rot_back_img = np.loadtxt(path_test + 'mnist_all_background_images_rotation_normalized_test.amat')

We have to trim the data set to have the correct size.

In [14]:
x_test = test_normal[:,:-1]
y_test = test_normal[:,-1]

x_test_rotated = test_rotated[:,:-1]
y_test_rotated = test_rotated[:,-1]

x_test_back_rand = test_back_rand[:,:-1]
y_test_back_rand = test_back_rand[:,-1]

x_test_back_img = test_back_img[:,:-1]
y_test_back_img = test_back_img[:,-1]

x_test_rot_back_img = test_rot_back_img[:,:-1]
y_test_rot_back_img = test_rot_back_img[:,-1]

In [8]:
print(x_test.shape, y_test.shape)
print(x_test_rotated.shape, y_test_rotated.shape)
print(x_test_back_rand.shape, y_test_back_rand.shape)
print(x_test_back_img.shape, y_test_back_img.shape)
print(x_test_rot_back_img.shape, y_test_rot_back_img.shape)

(12000, 784) (12000,)
(12000, 784) (12000,)
(12000, 784) (12000,)
(12000, 784) (12000,)
(12000, 784) (12000,)


## Parameters used for the Neural Network

For our model we are using the parameters as proposed in the original paper.

The __fix_params__ are the paramters that describe the input size of the image and the output size that we want to have. In this case we want an output that can have 10 different possibilities ( from 0 to 9 ).

__params__ are the parameters that define our convolutional layers and the size of the capsules in the capsule layer. We use two convolutions and the parameters for those convolutions have the suffix conv1 or conv2.

__train_params__ include all the training parameters used in the training of our Neural Network.


In [17]:
fix_params = {'tensor_width': 28, 'tensor_height': 28, 'tensor_depth': 1, 'n_output':10}

params = {'n_filters': 256, 
          'strides_conv1': 1, 'strides_conv2': 2,
         'kernel_size_conv1': 9, 'kernel_size_conv2': 9,
         'caps1_n_maps': 32, 'caps1_n_dims': 8,
         'caps2_n_dims': 16,
         'init_sigma':0.1}

train_params= {'m_plus': 0.9, 'm_minus': 0.1, 'lambda_': 0.5, 'mu': 0.5,
                  'n_hidden1':512, 'n_hidden2': 1024,
              'alpha': 0.005,
              'n_epochs' : 100, 'batch_size' :2,
              'drop_out_conv1':1, 'drop_out_conv2':1}

## CapsNet Model

We included a saver and Tensorboard to analyze the different parameters.
We put the Model into a function with the parameters as input to use make it easier to change different parameters inside the model for developing purposes.

In [18]:
def capsnetmodel_tb_save(train_parameters, fix_parameters, parameters):
    """
    This is the model that takes the different parametes as input. 
    Our data is always the same for our case hence we do not have it in our arguments.

    """
    print('Start Variables')

    # Fix parameters are loaded
    n_output = fix_parameters['n_output']
    tensor_width = fix_parameters['tensor_width']
    tensor_height = fix_parameters['tensor_height']
    tensor_depth = fix_parameters['tensor_depth']

    # Optimizable parameters are loaded
    n_filters = parameters['n_filters']
    strides_conv1 = parameters['strides_conv1']
    strides_conv2 = parameters['strides_conv2']
    kernel_size_conv1 = parameters['kernel_size_conv1']
    kernel_size_conv2 = parameters['kernel_size_conv2']

    caps1_n_maps = parameters['caps1_n_maps']
    caps1_n_dims = parameters['caps1_n_dims']
    caps2_n_dims = parameters['caps2_n_dims']
    conv_1_dropout = train_parameters['drop_out_conv1']
    conv_2_dropout = train_parameters['drop_out_conv2']

    init_sigma = parameters['init_sigma']

    m_plus = train_parameters['m_plus']
    m_minus = train_parameters['m_minus']
    lambda_ = train_parameters['lambda_']
    mu = train_parameters['mu']

    # Decoder parameters are loaded

    n_hidden1 = train_parameters['n_hidden1']
    n_hidden2 = train_parameters['n_hidden2']
    n_output_flat = fix_parameters['tensor_height'] * \
        fix_parameters['tensor_width'] * fix_parameters['tensor_depth']
    alpha = train_parameters['alpha']

    # Training parameters are loaded

    n_epochs = train_parameters['n_epochs']
    batch_size_train = train_parameters['batch_size']
    n_iterations_per_epoch = len(x_train) // batch_size_train
    n_iterations_validation = len(x_test) // batch_size_train
    best_loss_val = np.infty
    best_loss_val_rotated = np.infty
    best_loss_val_back_rand = np.infty
    best_loss_val_back_img = np.infty
    best_loss_val_rot_back_img = np.infty

    # Path for the Save-Checkpoint
    checkpoint_path = "./board/capsule_checkpoint"

    print('Start Model')

    # Our placeholders are the input X and the output Y.
    X = tf.placeholder(tf.float32, shape=[
                       None, tensor_width * tensor_height * tensor_depth], name='Input')
    y = tf.placeholder(tf.float32, shape=[None, n_output], name='Output')

    # We need to reshape the image to have the correct dimensions.
    x_image = tf.reshape(
        X, [-1, tensor_width, tensor_height, tensor_depth], name='images')

    # We use helper functions to define our Convolutional Layers
    def variable_with_weight_decay(name, shape, stddev, wd):
        dtype = tf.float32
        var = variable_on_cpu(
            name, shape, tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
        if wd is not None:
            weight_decay = tf.multiply(
                tf.nn.l2_loss(var), wd, name='weight_loss')
            tf.add_to_collection('losses', weight_decay)
        return var

    def variable_on_cpu(name, shape, initializer):
        with tf.device('/cpu:0'):
            dtype = tf.float32
            var = tf.get_variable(
                name, shape, initializer=initializer, dtype=dtype)
        return var

    # The first convolutional Layer
    with tf.variable_scope('conv1') as scope:
        
        # We use the parameters above to define our kernel for the first convolutional layer
        kernel = variable_with_weight_decay('weights', shape=[
                                            kernel_size_conv1, kernel_size_conv1, tensor_depth, n_filters], stddev=5e-2, wd=0.0)
        
        conv = tf.nn.conv2d(
            x_image, kernel, [1, strides_conv1, strides_conv1, 1], padding='SAME')
        biases = variable_on_cpu(
            'biases', [n_filters], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        conv1 = tf.nn.dropout(conv1, keep_prob=conv_1_dropout)

    # The second convolutional Layer
    with tf.variable_scope('conv2') as scope:
        
        # We use the parameters above to define our kernel for the second convolutional layer
        kernel = variable_with_weight_decay(
            'weights', shape=[kernel_size_conv2, kernel_size_conv2, n_filters, n_filters], stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(
            conv1, kernel, [1, strides_conv2, strides_conv2, 1], padding='SAME')
        biases = variable_on_cpu(
            'biases', [n_filters], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        conv2 = tf.nn.dropout(conv2, keep_prob=conv_2_dropout)
        
    
    # We take the dimensions of the second convolutional layer to connect it correctly to our capsules layer
    dim = conv2.get_shape().as_list()

    # Number of capsules in our layer is the number of maps we want times the two dimension of our layer
    caps1_n_caps = caps1_n_maps * dim[1] * dim[2]
    
    # Reshape of the convolutional layers into a capusle layer with the number of capsules calculated in the previous line.
    caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                           name="caps1_raw")

    # Implementation of the Squash function used in the paper to normalize the output of the first capsule layer.
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keepdims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector

    caps1_output = squash(caps1_raw, name="caps1_output")
    
    caps2_n_caps = n_output
    
    # W is the Weight Matrix from the paper
    W_init = tf.random_normal(
        shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
        stddev=init_sigma, dtype=tf.float32, name="W_init")
    W = tf.Variable(W_init, name="W")

    batch_size = tf.shape(X)[0]
    W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")
    
    # To correctly multiply the weight matrix with the output of Capsule Layer 1 
    # we expand(duplicate) the tensor along its axis.
    caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                           name="caps1_output_expanded")
    caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                       name="caps1_output_tile")
    caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                                 name="caps1_output_tiled")

    # Now we can correclty multiply the Weight matrix and the output of the first capsule layer
    caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                                name="caps2_predicted")
    
    #Start of the Routing by Agreement Algorithm
    
    #Initialize the weights
    raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                           dtype=np.float32, name="raw_weights")
    

    # We use two rounds of routing. In our case those two rounds are hard coded.
    # Round 1
    routing_weights = tf.nn.softmax(
        raw_weights, axis=2, name="routing_weights")

    weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                       name="weighted_predictions")
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keepdims=True,
                                 name="weighted_sum")

    caps2_output_round_1 = squash(weighted_sum, axis=-2,
                                  name="caps2_output_round_1")
    
    # Round 2
    #Expand the output by the number of capsules in the first layers
    caps2_output_round_1_tiled = tf.tile(
        caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
        name="caps2_output_round_1_tiled")
    
    # Update of the weights for round 2 as explained in the paper
    agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                          transpose_a=True, name="agreement")

    raw_weights_round_2 = tf.add(raw_weights, agreement,
                                 name="raw_weights_round_2")
    
    # Similar to round 1
    routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                            axis=2,
                                            name="routing_weights_round_2")
    weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                               caps2_predicted,
                                               name="weighted_predictions_round_2")
    weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                         axis=1, keepdims=True,
                                         name="weighted_sum_round_2")
    caps2_output_round_2 = squash(weighted_sum_round_2,
                                  axis=-2,
                                  name="caps2_output_round_2")

    caps2_output = caps2_output_round_2
    
    # Special Norm Function to avoid division by 0
    def safe_norm(s, axis=-1, epsilon=1e-7, keepdims=False, name=None):
        with tf.name_scope(name, default_name="safe_norm"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keepdims=keepdims)
            return tf.sqrt(squared_norm + epsilon)

    # The output probabilities are given by normalizing the output of the last capsule layer after round 2
    y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

    y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")

    y_pred = tf.squeeze(y_proba_argmax, axis=[1, 2], name="y_pred")

    
    # After predicting y we calculate the Loss with the Loss function specified in the paper
    print('Start Loss')
    y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

    T = tf.one_hot(y, depth=caps2_n_caps, name="T")

    caps2_output_norm = safe_norm(caps2_output, axis=-2, keepdims=True,
                                  name="caps2_output_norm")

    present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                                  name="present_error_raw")
    present_error = tf.reshape(present_error_raw, shape=(-1, n_output),
                               name="present_error")

    absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                                 name="absent_error_raw")
    absent_error = tf.reshape(absent_error_raw, shape=(-1, n_output),
                              name="absent_error")
    
    # This is the actual Loss function
    L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
               name="L")

    margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")
    tf.summary.scalar('margin_loss', margin_loss)

    
    
    # Reconstruction of the input with the decoder
    #Our mask is the vector where only the correct label is True
    mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                                   name="mask_with_labels")

    reconstruction_targets = tf.cond(mask_with_labels,  # condition
                                     lambda: y,        # if True
                                     lambda: y_pred,   # if False
                                     name="reconstruction_targets")

    reconstruction_mask = tf.one_hot(reconstruction_targets,
                                     depth = caps2_n_caps,
                                     name="reconstruction_mask")

    reconstruction_mask_reshaped = tf.reshape(
        reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
        name="reconstruction_mask_reshaped")

    caps2_output_masked = tf.multiply(
        caps2_output, reconstruction_mask_reshaped,
        name="caps2_output_masked")

    decoder_input = tf.reshape(caps2_output_masked,
                               [-1, caps2_n_caps * caps2_n_dims],
                               name="decoder_input")

    # Decoder with three fully connected layers
    print('start decoder')
    with tf.name_scope("decoder"):
        hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                                  activation=tf.nn.relu,
                                  name="hidden1")
        hidden2 = tf.layers.dense(hidden1, n_hidden2,
                                  activation=tf.nn.relu,
                                  name="hidden2")
        decoder_output = tf.layers.dense(hidden2, n_output_flat,
                                         activation=tf.nn.sigmoid,
                                         name="decoder_output")

    # Reconstruction Loss function
    print('Start Reconstruction Loss')

    # Flatten X
    X_flat = tf.reshape(X, [-1, n_output_flat], name="X_flat")
    
    
    # We take the squared difference between the original input and the reconstructed one
    
    squared_difference = tf.square(X_flat - decoder_output,
                                   name="squared_difference")
    reconstruction_loss = tf.reduce_mean(squared_difference,
                                         name="reconstruction_loss")
    
    # The loss function of our model is composed of the  reconstruction loss function (regularizer) 
    # and first loss function.

    loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")
    tf.summary.scalar('loss', loss)

    correct = tf.equal(y, y_pred, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

    tf.summary.scalar('accuracy', accuracy)

    # We use the AdamOptimizer for this model and we want to minimize the Loss function specified earlier
    
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss, name="training_op")

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    print('Start Training')
    
    # All the summaries and graphs are saved to those paths.
    summaries_dir = "./board/summaries"

    graph_dir = './board/graphs'

    merged = tf.summary.merge_all()

    
    # Start of our tensorflow session 
    
    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(summaries_dir + '/train')
        test_writer = tf.summary.FileWriter(summaries_dir + '/test')
        test_writer_rotated = tf.summary.FileWriter(
            summaries_dir + '/test_rotated')
        test_writer_back_rand = tf.summary.FileWriter(
            summaries_dir + '/test_back_rand')
        test_writer_back_img = tf.summary.FileWriter(
            summaries_dir + '/test_back_img')
        test_writer_rot_back_img = tf.summary.FileWriter(
            summaries_dir + '/test_rot_back_img')

        graph_writer = tf.summary.FileWriter(graph_dir, sess.graph)

        if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
            saver.restore(sess, checkpoint_path)
            print("Starting from checkpoint")
        else:
            init.run()
            print("Starting from scratch")
            
            
            
        # Training of the model starts here.
        
        for epoch in range(n_epochs):
            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('TRAINING EPOCH: {}'.format(epoch))
            acc_trains = []
            for iteration in range(1, n_iterations_per_epoch + 1):
                
                X_batch = x_train[(iteration-1) *
                                  batch_size_train:iteration*batch_size_train]
                y_batch = y_train[(iteration-1) *
                                  batch_size_train:iteration*batch_size_train]
                
                # Run the training operation and measure the loss:

                summary, _, loss_train, acc_tra = sess.run(
                    [merged, training_op, loss, accuracy],
                    feed_dict={X: X_batch,
                               y: y_batch,
                               mask_with_labels: True})
                train_writer.add_summary(
                    summary, iteration + n_iterations_per_epoch*epoch)
                acc_trains.append(acc_tra)
                print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                    iteration, n_iterations_per_epoch,
                    iteration * 100 / n_iterations_per_epoch,
                    loss_train),
                    end="")
            acc_trains = np.average(acc_trains)
            print('\nTrain Accuracy {:.4f}%'.format(acc_trains*100))

            
            
            # Testing of the normal MNIST starts here
            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('TESTING EPOCH: {}'.format(epoch))

            # At the end of each epoch,
            # measure the validation loss and accuracy:
            
            loss_vals = []
            acc_vals = []
            outputs = np.zeros([])
            
            for iteration in range(1, n_iterations_validation + 1):
                X_batch = x_test[(iteration-1) *
                                 batch_size_train:iteration*batch_size_train]
                y_batch = y_test[(iteration-1) *
                                 batch_size_train:iteration*batch_size_train]

                summary_test, loss_val, acc_val, output = sess.run(
                    [merged, loss, accuracy, y_pred],
                    feed_dict={X: X_batch,
                               y: y_batch})

                test_writer.add_summary(
                    summary_test, iteration + n_iterations_per_epoch*epoch)

                loss_vals.append(loss_val)
                acc_vals.append(acc_val)
                outputs = np.append(outputs, output)
                
                print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                    iteration * 100 / n_iterations_validation),
                    end=" " * 10)
            loss_val = np.mean(loss_vals)
            acc_val = np.mean(acc_vals)
            print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
                epoch + 1, acc_val * 100, loss_val,
                " (improved)" if loss_val < best_loss_val else ""))

            # And save the model if it improved:
            if loss_val < best_loss_val:
                save_path = saver.save(sess, checkpoint_path)
                best_loss_val = loss_val


#############
# Test model with rotated MNIST
################

            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('ROTATED TESTING EPOCH: {}'.format(epoch))

            # At the end of each epoch,
            # measure the validation loss and accuracy:
            loss_vals_rotated = []
            acc_vals_rotated = []
            outputs_rotated = np.zeros([])
            for iteration in range(1, n_iterations_validation + 1):
                X_batch = x_test_rotated[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]
                y_batch = y_test_rotated[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]

                summary_test_rotated, loss_val_rotated, acc_val_rotated, output_rotated = sess.run(
                    [merged, loss, accuracy, y_pred],
                    feed_dict={X: X_batch,
                               y: y_batch})

                test_writer_rotated.add_summary(
                    summary_test_rotated, iteration + n_iterations_per_epoch*epoch)

                loss_vals_rotated.append(loss_val_rotated)
                acc_vals_rotated.append(acc_val_rotated)
                outputs_rotated = np.append(outputs_rotated, output_rotated)
                print("\rEvaluating the model_rotated: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                    iteration * 100 / n_iterations_validation),
                    end=" " * 10)
            loss_val_rotated = np.mean(loss_vals_rotated)
            acc_val_rotated = np.mean(acc_vals_rotated)
            print("\rEpoch: {}  Val accuracy_rotated: {:.4f}%  Loss_rotated: {:.6f}{}".format(
                epoch + 1, acc_val_rotated * 100, loss_val_rotated,
                " (improved rotation)" if loss_val_rotated < best_loss_val_rotated else ""))


#############
# Test model with MNIST with a random background
################

            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('RANDOM BACK TESTING EPOCH: {}'.format(epoch))

            # At the end of each epoch,
            # measure the validation loss and accuracy:
            loss_vals_back_rand = []
            acc_vals_back_rand = []
            outputs_back_rand = np.zeros([])
            for iteration in range(1, n_iterations_validation + 1):
                X_batch = x_test_back_rand[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]
                y_batch = y_test_back_rand[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]

                summary_test_back_rand, loss_val_back_rand, acc_val_back_rand, output_back_rand = sess.run(
                    [merged, loss, accuracy, y_pred],
                    feed_dict={X: X_batch,
                               y: y_batch})

                test_writer_back_rand.add_summary(
                    summary_test_back_rand, iteration + n_iterations_per_epoch*epoch)

                loss_vals_back_rand.append(loss_val_back_rand)
                acc_vals_back_rand.append(acc_val_back_rand)
                outputs_back_rand = np.append(
                    outputs_back_rand, output_back_rand)
                print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                    iteration * 100 / n_iterations_validation),
                    end=" " * 10)
            loss_val_back_rand = np.mean(loss_vals_back_rand)
            acc_val_back_rand = np.mean(acc_vals_back_rand)
            print("\rEpoch: {}  Val accuracy_back_rand: {:.4f}%  Loss_back_rand: {:.6f}{}".format(
                epoch + 1, acc_val_back_rand * 100, loss_val_back_rand,
                " (improved random back)" if loss_val_back_rand < best_loss_val_back_rand else ""))


#############
# Test model with MNIST with an image as Background
################

            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('BACK IMG TESTING EPOCH: {}'.format(epoch))

            # At the end of each epoch,
            # measure the validation loss and accuracy:
            loss_vals_back_img = []
            acc_vals_back_img = []
            outputs_back_img = np.zeros([])
            for iteration in range(1, n_iterations_validation + 1):
                # .reshape([-1,tensor_depth,tensor_width, tensor_height]).transpose([0,3,2,1]).reshape(-1, tensor_depth*tensor_width*tensor_height)
                X_batch = x_test_back_img[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]
                y_batch = y_test_back_img[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]

                summary_test_back_img, loss_val_back_img, acc_val_back_img, output_back_img = sess.run(
                    [merged, loss, accuracy, y_pred],
                    feed_dict={X: X_batch,
                               y: y_batch})

                test_writer_back_img.add_summary(
                    summary_test_back_img, iteration + n_iterations_per_epoch*epoch)

                loss_vals_back_img.append(loss_val_back_img)
                acc_vals_back_img.append(acc_val_back_img)
                outputs_back_img = np.append(outputs_back_img, output_back_img)
                print("\rEvaluating the model_back_img: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                    iteration * 100 / n_iterations_validation),
                    end=" " * 10)
            loss_val_back_img = np.mean(loss_vals_back_img)
            acc_val_back_img = np.mean(acc_vals_back_img)
            print("\rEpoch: {}  Val accuracy_back_img: {:.4f}%  Loss_back_img: {:.6f}{}".format(
                epoch + 1, acc_val_back_img * 100, loss_val_back_img,
                " (improved back image)" if loss_val_back_img < best_loss_val_back_img else ""))


#############
# Test of the model with a rotated MNIST set that has an image as background
################

            print('--------------------------------------------------------------')
            print('--------------------------------------------------------------')
            print('ROT BACK IMG TESTING EPOCH: {}'.format(epoch))

            # At the end of each epoch,
            # measure the validation loss and accuracy:
            loss_vals_rot_back_img = []
            acc_vals_rot_back_img = []
            outputs_rot_back_img = np.zeros([])
            for iteration in range(1, n_iterations_validation + 1):
                # .reshape([-1,tensor_depth,tensor_width, tensor_height]).transpose([0,3,2,1]).reshape(-1, tensor_depth*tensor_width*tensor_height)
                X_batch = x_test_rot_back_img[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]
                y_batch = y_test_rot_back_img[(
                    iteration-1)*batch_size_train:iteration*batch_size_train]

                summary_test_rot_back_img, loss_val_rot_back_img, acc_val_rot_back_img, output_rot_back_img = sess.run(
                    [merged, loss, accuracy, y_pred],
                    feed_dict={X: X_batch,
                               y: y_batch})

                test_writer_rot_back_img.add_summary(
                    summary_test_rot_back_img, iteration + n_iterations_per_epoch*epoch)

                loss_vals_rot_back_img.append(loss_val_rot_back_img)
                acc_vals_rot_back_img.append(acc_val_rot_back_img)
                outputs_rot_back_img = np.append(
                    outputs_rot_back_img, output_rot_back_img)
                print("\rEvaluating the model_rot_back_img: {}/{} ({:.1f}%)".format(
                    iteration, n_iterations_validation,
                    iteration * 100 / n_iterations_validation),
                    end=" " * 10)
            loss_val_rot_back_img = np.mean(loss_vals_rot_back_img)
            acc_val_rot_back_img = np.mean(acc_vals_rot_back_img)
            print("\rEpoch: {}  Val accuracy_rot_back_img: {:.4f}%  Loss_rot_back_img: {:.6f}{}".format(
                epoch + 1, acc_val_rot_back_img * 100, loss_val_rot_back_img,
                " (improved rota back img)" if loss_val_rot_back_img < best_loss_val_rot_back_img else ""))

        train_writer.close()

        graph_writer.close()
        test_writer_rotated.close()
        test_writer_back_rand.close()
        test_writer_back_img.close()
        test_writer_rot_back_img.close()

## Execution of the Model

In [19]:
capsnetmodel_tb_save(train_params, fix_params, params)

Start Variables
Start Model
Start Loss
start decoder
Start Reconstruction Loss
Start Training
Starting from scratch
--------------------------------------------------------------
--------------------------------------------------------------
TRAINING EPOCH: 0
Iteration: 42/27500 (0.2%)  Loss: 0.43434

KeyboardInterrupt: 